In [ ]:
%reload_ext autoreload
%autoreload 2

import functools
print = functools.partial(print, flush=True)

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import pickle

import flexiznam as flz
from cottage_analysis.io_module import harp
from cottage_analysis.preprocessing import find_frames
from cottage_analysis.imaging.common import find_frames as find_img_frames
from cottage_analysis.filepath import generate_filepaths
from cottage_analysis.imaging.common import imaging_loggers_formatting as format_loggers
from cottage_analysis.preprocessing import synchronisation
from cottage_analysis.analysis import find_depth_neurons, fit_gaussian_blob
from cottage_analysis.stimulus_structure import spheres_tube

In [ ]:
# Example session
project = 'hey2_3d-vision_foodres_20220101'
mouse = 'PZAH8.2f'
session = 'S20230126'
RECORDING = 'R144331_SpheresPermTubeReward'
protocol = 'SpheresPermTubeReward'
MESSAGES = 'harpmessage.bin'
flexilims_session = flz.get_flexilims_session(project_id=project)
# all_protocol_recording_entries = generate_filepaths.get_all_recording_entries(project=project, 
#                                                                               mouse=mouse, 
#                                                                               session=session, 
#                                                                               protocol=protocol, 
#                                                                               flexilims_session=flexilims_session)

# # DO NOT RUN THIS FUNCTION (TAKES 2hrs ish): to find monitor frames from photodiode signal
# find_monitor_frames(project=project, 
#                     mouse=mouse, 
#                     session=session, 
#                     protocol=protocol, 
#                     all_protocol_recording_entries=None, 
#                     irecording=None, 
#                     flexilims_session=None)
    

In [ ]:
# Generate synchronisation dataframes
vs_df =  synchronisation.generate_vs_df(project=project, 
                                        mouse=mouse, 
                                        session=session, 
                                        protocol=protocol, 
                                        irecording=0)
trials_df, imaging_df = synchronisation.generate_trials_df(project=project, 
                                                           mouse=mouse, 
                                                           session=session, 
                                                           protocol=protocol, 
                                                           vs_df=vs_df, 
                                                           irecording=0)

In [ ]:
# Find depth neurons and fit preferred depth
neurons_df = find_depth_neurons.find_depth_neurons(
    project=project, 
    mouse=mouse, 
    session=session, 
    protocol="SpheresPermTubeReward", 
    rs_thr=0.2
)

neurons_df = find_depth_neurons.fit_preferred_depth(
    project=project,
    mouse=mouse,
    session=session,
    protocol="SpheresPermTubeReward",
    depth_min=0.02,
    depth_max=20,
    batch_num=10,
)

In [ ]:
neurons_df

In [ ]:
# Fit gaussian blob to neuronal activity (THIS WILL ALSO TAKE QUITE LONG! SO STOP EARLIER AND CHECK A FEW CELLS)
neurons_df = fit_gaussian_blob.fit_gaussian_blob(project=project, 
                  mouse=mouse, 
                  session=session, 
                  protocol="SpheresPermTubeReward", 
                  rs_thr=0.01, 
                  param_range={'rs_min':0.005, 'rs_max':5, 'of_min':0.03, 'of_max':3000}, 
                  batch_num=1)

In [ ]:
# Regenerate sphere stimuli
(
    rawdata_folder,
    _,
    _,
    _,
    _,
) = generate_filepaths.generate_file_folders(
    project=project,
    mouse=mouse,
    session=session,
    protocol=protocol,
    all_protocol_recording_entries=None,
    recording_no=0,
)

param_log = pd.read_csv(rawdata_folder / "NewParams.csv")
param_log = param_log.rename(
    columns={"Radius": "Depth"}
)

output = spheres_tube.regenerate_frames(
    frame_times=imaging_df['harptime_imaging_trigger'].values,
    trials_df=trials_df,
    vs_df=vs_df,
    param_logger=param_log,
    time_column="HarpTime",
    resolution=1,
    sphere_size=10,
    azimuth_limits=(-120, 120),
    elevation_limits=(-40, 40),
    verbose=True,
    output_datatype="int16",
    output=None,
)
plt.imshow(output[708,:,:], origin='lower')